<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# 1. magic for inline plot
# 2. magic to print version
# 3. magic so that the notebook will reload external python modules
# 4. magic to enable retina (high resolution) plots
# https://gist.github.com/minrk/3301035
%matplotlib inline
%load_ext watermark
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# change default style figure and font size
plt.rcParams['figure.figsize'] = 8, 6
plt.rcParams['font.size'] = 12

%watermark -a 'Ethen' -d -t -v -p numpy,pandas,sklearn,matplotlib

Ethen 2018-12-05 20:18:58 

CPython 3.6.4
IPython 6.4.0

numpy 1.14.1
pandas 0.23.0
sklearn 0.19.1
matplotlib 2.2.2


- https://techcrunch.com/2017/03/29/similarity-search/

In [31]:
import nanopq
import numpy as np
from sklearn.preprocessing import normalize

N, Nt, D = 10000, 2000, 1024
X = np.random.random((N, D)).astype(np.float32)  # 10,000 128-dim vectors to be indexed
X = normalize(X)
Xt = np.random.random((Nt, D)).astype(np.float32)  # 2,000 128-dim vectors for training
Xt = normalize(Xt)
query = np.random.random((D,)).astype(np.float32)  # a 128-dim query vector

# Instantiate with M=8 sub-spaces
pq = nanopq.PQ(M=8)

# Train codewords
pq.fit(Xt) 

M: 8, Ks: 256, code_dtype: <class 'numpy.uint8'>
iter: 20, seed: 123
Training the subspace: 0 / 8
Training the subspace: 1 / 8
Training the subspace: 2 / 8
Training the subspace: 3 / 8
Training the subspace: 4 / 8
Training the subspace: 5 / 8
Training the subspace: 6 / 8
Training the subspace: 7 / 8


In [32]:
# Encode to PQ-codes
X_code = pq.encode(X)  # (10000, 8) with dtype=np.uint8

# Results: create a distance table online, and compute Asymmetric Distance to each PQ-code 
dists = pq.dtable(query).adist(X_code)  # (10000, )
dists[:30]

Encoding the subspace: 0 / 8
Encoding the subspace: 1 / 8
Encoding the subspace: 2 / 8
Encoding the subspace: 3 / 8
Encoding the subspace: 4 / 8
Encoding the subspace: 5 / 8
Encoding the subspace: 6 / 8
Encoding the subspace: 7 / 8


array([311.044  , 311.51318, 311.4721 , 311.32654, 311.30542, 311.22128,
       311.4214 , 311.23724, 311.40814, 311.43896, 311.24695, 311.21133,
       310.98718, 311.41983, 311.1068 , 311.5326 , 311.2611 , 311.37054,
       311.74463, 311.24646, 311.40475, 311.24908, 311.44696, 311.15454,
       311.15988, 311.28583, 311.5368 , 311.36322, 310.90143, 311.2066 ],
      dtype=float32)

In [33]:
np.argsort(dists)

array([ 889, 6889, 9964, ..., 2767, 8743, 7357])

In [34]:
dists_exact = np.linalg.norm(X - query, axis=1) ** 2
print(dists_exact[:30])

[310.994   311.5402  312.03064 311.7561  311.56564 311.13788 311.6032
 311.37485 311.72223 311.82077 311.38724 311.18155 311.86096 311.3233
 311.5676  311.18518 311.22064 311.47934 311.80896 311.1381  311.81024
 311.4678  311.77838 311.38797 310.91553 311.7592  311.2777  312.35126
 311.27267 311.6474 ]


In [35]:
np.argsort(dists_exact)

array([4761, 7390, 8903, ..., 5883, 5321, 3242])